In [1]:
import os, sys
sys.path.append('../functions')

In [2]:
% matplotlib inline
import pandas
import numpy as np
import matplotlib.pyplot as plt

# Reading data

In [3]:
all_mctracks = pandas.read_csv('Data/MCTrack.csv', delimiter=',')
all_velo_points = pandas.read_csv('Data/vetoPoint.csv', delimiter=',')
all_hits = pandas.read_csv('Data/strawtubesPoint_decoded_hits.csv', delimiter=',')
all_hits['Index'] = all_hits.index
all_hits = all_hits[['Index', 'X', 'Y', 'Z', 'TrackID', 'Time', 'StatNb', 'ViewNb', 'PlaneNb', 'LayerNb', 'StrawNb', 'dist2Wire', 'event', 'k']]
all_hits.head()

,Index,X,Y,Z,TrackID,Time,StatNb,ViewNb,PlaneNb,LayerNb,StrawNb,dist2Wire,event,k
0,0,8.883409,-61.202305,-1974.859985,3,232.044861,5,0,0,0,319,0.274822,0,0
1,1,8.858191,-61.225544,-1973.742188,3,232.074081,5,0,0,1,320,0.582455,0,1
2,2,8.824563,-61.256523,-1972.251953,3,232.118286,5,0,1,0,319,0.111252,0,2
3,3,8.632817,-61.432972,-1963.756592,3,232.406479,5,1,0,1,320,0.164412,0,3
4,4,8.573978,-61.487133,-1961.149292,3,232.488541,5,1,1,1,320,0.226827,0,4


In [39]:
from ReconstructibleTracks import get_reconstractible_tracks

reconstructible_events = {}

for event_id in range(2500, 5000):
    
    tracks = get_reconstractible_tracks(event_id, all_hits, all_mctracks, all_velo_points)
    
    if tracks != []:
        reconstructible_events[event_id] = tracks

../functions/ReconstructibleTracks.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  10000000 * hits.StatNb.values


In [44]:
#import cPickle as pickle

#reconstructible_events_pickle = open('reconstructible_events', 'wb')
#pickle.dump(reconstructible_events, reconstructible_events_pickle)
#reconstructible_events_pickle.close()

In [4]:
import cPickle as pickle

reconstructible_events_pickle2 = open('reconstructible_events', 'rb')
reconstructible_events = pickle.load(reconstructible_events_pickle2)
reconstructible_events_pickle2.close()

In [5]:
from WeightedRegression import *

In [6]:
import UV_views

def get_zx_hits(event, zy_plane_k, zy_plane_b):
    
    stereo_hits = UV_views.modify_for_xz_analysis_1_2(event)
    
    stereo_hits['Wy'] = zy_plane_k * stereo_hits.Wz.values + zy_plane_b
    stereo_hits['Wx'] = (stereo_hits.Wx2.values - stereo_hits.Wx1.values) / (stereo_hits.Wy2.values - stereo_hits.Wy1.values) * \
                        (stereo_hits.Wy.values - stereo_hits.Wy1.values) + stereo_hits.Wx1.values
    
    return stereo_hits#[numpy.abs(stereo_hits.Wx.values) < 300]

# Y-Data Generation

In [9]:
from Y_views_new import *

def get_y_view_data(all_hit_ids, reconstructible_events_ids):

    min_dists = []
    max_dists = []
    scores = []
    score_stds =[]
    labels = []
    ks = []
    bs = []
    lens = []
    
    counter = 0


    for event_id in reconstructible_events_ids:
        
        # get all event hits
        event = all_hits[all_hits['event'] == event_id]
        
        # get hits in y_views
        event_y = modify_for_yz_analysis_1_2(event)
        event_y = event_y[ (event_y.TrackID == 2) + (event_y.TrackID == 3) ]

        
        for track_id in [2.0, 3.0]:
            
            # get hits for the track and other hists
            track = event_y[event_y.TrackID == track_id]
            false_track = event_y[event_y.TrackID != track_id]
            
            if len(track) == 0:
                continue

            # mix hits from two tracks
            mixed_track = pandas.concat([track[track.StatNb == 1], false_track[false_track.StatNb == 2]], axis=0)

            if len(track.values) == 0:
                print event_id, track_id
                continue

            if len(false_track.values) == 0:
                print event_id, track_id
                continue

            if len(mixed_track.values) == 0:
                print event_id, track_id
                continue


            # huts labels
            pos_labels = [1]
            neg_labels = [0]

            # fit true fit
            lr = WeightedRegression()
            lr.fit(track)
            track_dists = lr.dist2track(track)[:, 0]
            score = np.mean(track_dists)
            score_std = np.std(track_dists)
                        
            # fit mixed track
            mixed_lr = WeightedRegression()
            mixed_lr.fit(mixed_track)
            mixed_track_dists = mixed_lr.dist2track(mixed_track)[:, 0]
            mixed_score = np.mean(mixed_track_dists)
            mixed_score_std = np.std(mixed_track_dists)
        
            min_dists += [np.min(track_dists)] + [np.min(mixed_track_dists)]
            max_dists += [np.max(track_dists)] + [np.max(mixed_track_dists)]
            lens += [len(track)] + [len(mixed_track)]
            # collect all data
            scores += [score] + [mixed_score]
            score_stds += [score_std] + [mixed_score_std]

            labels += pos_labels + neg_labels
            
            ks += [lr.lr.coef_[0][0]] + [mixed_lr.lr.coef_[0][0]]
                
            bs += [lr.lr.intercept_[0]] + [mixed_lr.lr.intercept_[0]]
            
        counter += 1
        #if counter % 10 == 0:
        print 'Counter = ', counter, ' / ', len(reconstructible_events_ids)


    data = pandas.DataFrame()
    data['min_dist'] = min_dists
    data['max_dist'] = max_dists
    data['score_std'] = score_stds
    data['score'] = scores
    data['slope'] = ks
    data['intercept'] = bs
    data['label'] = labels
    data['len'] = lens
    
    return data

In [101]:
data_y = get_y_view_data(all_hits, reconstructible_events)

Counter =  1  /  362
Counter =  2  /  362
Counter =  3  /  362
Counter =  4  /  362
Counter =  5  /  362
Counter =  6  /  362
Counter =  7  /  362
Counter =  8  /  362
Counter =  9  /  362
Counter =  10  /  362
Counter =  11  /  362
Counter =  12  /  362
Counter =  13  /  362
Counter =  14  /  362
Counter =  15  /  362
Counter =  16  /  362
Counter =  17  /  362
Counter =  18  /  362
Counter =  19  /  362
Counter =  20  /  362
Counter =  21  /  362
Counter =  22  /  362
Counter =  23  /  362
Counter =  24  /  362
Counter =  25  /  362
Counter =  26  /  362
Counter =  27  /  362
Counter =  28  /  362
Counter =  29  /  362
Counter =  30  /  362
Counter =  31  /  362
Counter =  32  /  362
Counter =  33  /  362
Counter =  34  /  362
Counter =  35  /  362
Counter =  36  /  362
4297 3.0
Counter =  37  /  362
Counter =  38  /  362
Counter =  39  /  362
Counter =  40  /  362
Counter =  41  /  362
Counter =  42  /  362
Counter =  43  /  362
Counter =  44  /  362
Counter =  45  /  362
Counter = 

In [102]:
import cPickle as pickle

data_y_pickle = open('data_y', 'wb')
pickle.dump(data_y, data_y_pickle)
data_y_pickle.close()

In [32]:
import cPickle as pickle

data_y_pickle2 = open('data_y', 'rb')
data_y = pickle.load(data_y_pickle2)
data_y_pickle2.close()

# Train & Test Y-Classifier

In [33]:
from sklearn import cross_validation
data_y_train, data_y_test, label_train, label_test = cross_validation.train_test_split(data_y.drop(['label'],axis=1),\
                                                                                       data_y[['label']],\
                                                                                       test_size = 0.3)

In [34]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clf_y = RandomForestClassifier(n_estimators=1000, 
                               max_features='auto', 
                               max_depth=None, 
                               bootstrap=False, 
                               class_weight='balanced',
                               n_jobs = 3,
                               criterion = 'gini',
                               min_samples_leaf = 1)
#clf_y = GradientBoostingClassifier(n_estimators=1000, 
#                                   learning_rate=0.01, 
#                                   max_depth=6, 
#                                   max_features='auto', 
#                                   subsample=0.5,
#                                   min_samples_split=10)

clf_y.fit(data_y_train.values, label_train.values)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [35]:
clf_y.score(data_y_test.values, label_test.values)

0.99058823529411766

In [36]:
clf_y.feature_importances_

array([ 0.13039085,  0.26138196,  0.19558229,  0.34673606,  0.04539705,
        0.01883355,  0.00167823])

In [37]:
import cPickle as pickle

clf_y_pickle = open('clf_y.pkl', 'wb')
pickle.dump(clf_y, clf_y_pickle)
clf_y_pickle.close()

In [38]:
import cPickle as pickle

clf_y_pickle2 = open('clf_y.pkl', 'rb')
clf_y = pickle.load(clf_y_pickle2)
clf_y_pickle2.close()

In [129]:
data_y.drop(['label'], axis=1).head()

,min_dist,max_dist,score_std,score,slope,intercept,len
0,0.002534,0.035389,0.011270,0.017120,0.010082,-44.668349,9
1,2.917088,5.640999,0.761454,4.110658,-0.295412,748.248885,8
2,0.002889,0.010743,0.002706,0.007917,-0.012553,-43.637905,8
3,0.711652,6.318016,2.259685,3.497472,0.276475,-792.182167,9
4,0.000414,0.008912,0.002906,0.003614,-0.012532,-44.862743,9


# Stereo-Data Generation

In [8]:
from Y_views_new import *
import UV_views

def get_stereo_view_data(all_hit_ids, reconstructible_events_ids):
    
    min_dists = []
    max_dists = []
    scores = []
    score_stds =[]
    labels = []
    ks = []
    bs = []
    lens = []
    
    counter = 0


    for event_id in reconstructible_events_ids:
        
        # get all event hits
        event = all_hits[all_hits['event'] == event_id]
        
        # get hits in y_views
        event_y = modify_for_yz_analysis_1_2(event)
        event_y = event_y[ (event_y.TrackID == 2) + (event_y.TrackID == 3) ]
        
        # get hits in stereo_views
        event_stereo = UV_views.modify_for_xz_analysis_1_2(event)
        event_stereo = event_stereo[ (event_stereo.TrackID == 2) + (event_stereo.TrackID == 3) ]

        
        for track_id in [2.0, 3.0]:
            
            # get hits for the track and other hists
            track = event_y[event_y.TrackID == track_id]
            false_track = event_y[event_y.TrackID != track_id]

            if len(track) == 0:
                continue

            # mix hits from two tracks
            mixed_track = pandas.concat([track[track.StatNb == 1], false_track[false_track.StatNb == 2]], axis=0)

            if len(track.values) == 0:
                print event_id, track_id
                continue

            if len(false_track.values) == 0:
                print event_id, track_id
                continue

            if len(mixed_track.values) == 0:
                print event_id, track_id
                continue

            # fit true fit in y_views
            lr = WeightedRegression()
            lr.fit(track)
            
            stereo_hits = get_zx_hits(event, lr.lr.coef_[0][0], lr.lr.intercept_[0])
            stereo_track = stereo_hits[stereo_hits.TrackID == track_id]
            stereo_track_false = stereo_hits[stereo_hits.TrackID != track_id]
            
            if len(stereo_track.values) == 0:
                print 'stereo_track', event_id, track_id
                continue
                
            if len(stereo_track_false.values) == 0:
                print 'stereo_track_false', event_id, track_id
                continue
            
            # fit mixed track in y_views
            mixed_lr = WeightedRegression()
            mixed_lr.fit(mixed_track)
            
            stereo_mixed_hits = get_zx_hits(event, mixed_lr.lr.coef_[0][0], mixed_lr.lr.intercept_[0])
            stereo_mixed_track = pandas.concat([stereo_mixed_hits[(stereo_mixed_hits.TrackID == track_id) * (stereo_mixed_hits.StatNb == 1)],
                                                stereo_mixed_hits[(stereo_mixed_hits.TrackID != track_id) * (stereo_mixed_hits.StatNb == 2)]],
                                                axis=0)
            
            if len(stereo_mixed_track.values) == 0:
                print 'stereo_mixed_track', event_id, track_id
                continue
            
            
            # fit true fit in stereo_views
            lr_stereo = WeightedRegression(view='Stereo')
            lr_stereo.fit(stereo_track)
            stereo_track_dists = lr_stereo.dist2track(stereo_track)
            score = np.mean(stereo_track_dists)
            score_std = np.std(stereo_track_dists)
            
            # fit mixed track in stereo_views
            mixed_lr_stereo = WeightedRegression(view='Stereo')
            mixed_lr_stereo.fit(stereo_mixed_track)
            mixed_stereo_track_dists = mixed_lr_stereo.dist2track(stereo_mixed_track)
            mixed_score = np.mean(mixed_stereo_track_dists)
            mixed_score_std = np.std(mixed_stereo_track_dists)
            
            
            # huts labels
            pos_labels = [1]
            neg_labels = [0]
            labels += pos_labels + neg_labels
        
            # collect all data
            scores += [score] + [mixed_score]
            score_stds += [score_std] + [mixed_score_std]
            min_dists += [np.min(stereo_track_dists)] + [np.min(mixed_stereo_track_dists)]
            max_dists += [np.max(stereo_track_dists)] + [np.max(mixed_stereo_track_dists)]
            
            ks += [lr_stereo.lr.coef_[0][0]] + [mixed_lr_stereo.lr.coef_[0][0]]
                
            bs += [lr_stereo.lr.intercept_[0]] + [mixed_lr_stereo.lr.intercept_[0]]
            
            lens += [len(stereo_track_dists)] + [len(mixed_stereo_track_dists)]
            
        counter += 1
        #if counter % 10 == 0:
        print 'Counter = ', counter, ' / ', len(reconstructible_events_ids)


    data = pandas.DataFrame()
    data['min_dist'] = min_dists
    data['max_dist'] = max_dists
    data['score_std'] = score_stds
    data['score'] = scores
    data['slope'] = ks
    data['intercept'] = bs
    data['label'] = labels
    data['len'] = lens
    
    return data

In [9]:
data_stereo = get_stereo_view_data(all_hits, reconstructible_events)

/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python2.7/site-packages/pandas/computation/expressions.py:190: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


Counter =  1  /  362
Counter =  2  /  362
Counter =  3  /  362
Counter =  4  /  362
Counter =  5  /  362
Counter =  6  /  362
Counter =  7  /  362
Counter =  8  /  362
Counter =  9  /  362
Counter =  10  /  362
Counter =  11  /  362
Counter =  12  /  362
Counter =  13  /  362
Counter =  14  /  362
Counter =  15  /  362
Counter =  16  /  362
Counter =  17  /  362
Counter =  18  /  362
Counter =  19  /  362
Counter =  20  /  362
Counter =  21  /  362
Counter =  22  /  362
Counter =  23  /  362
Counter =  24  /  362
Counter =  25  /  362
Counter =  26  /  362
Counter =  27  /  362
Counter =  28  /  362
Counter =  29  /  362
Counter =  30  /  362
Counter =  31  /  362
Counter =  32  /  362
Counter =  33  /  362
Counter =  34  /  362
Counter =  35  /  362
Counter =  36  /  362
4297 3.0
Counter =  37  /  362
Counter =  38  /  362
Counter =  39  /  362
Counter =  40  /  362
Counter =  41  /  362
Counter =  42  /  362
Counter =  43  /  362
Counter =  44  /  362
Counter =  45  /  362
Counter = 

/usr/local/lib/python2.7/site-packages/pandas/computation/expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


In [13]:
data_stereo.head()

,min_dist,max_dist,score_std,score,slope,intercept,label,len
0,0.007302,0.334408,0.125418,0.196261,-0.015163,63.108224,1,10
1,1.259619,9.913027,2.782271,4.891600,0.627255,-1582.378602,0,10
2,0.017426,0.269795,0.104010,0.158463,0.023567,84.801992,1,9
3,1.058307,43.673853,14.277642,18.732651,-0.493514,1406.961035,0,9
4,0.031446,0.460475,0.194752,0.275425,-0.006853,-93.787694,1,9


In [11]:
import cPickle as pickle

data_stereo_pickle = open('data_stereo', 'wb')
pickle.dump(data_stereo, data_stereo_pickle)
data_stereo_pickle.close()

In [39]:
import cPickle as pickle

data_stereo_pickle2 = open('data_stereo', 'rb')
data_stereo = pickle.load(data_stereo_pickle2)
data_stereo_pickle2.close()

# Train & Test Strereo-Classifier

In [26]:
from sklearn import cross_validation
data_stereo_train, data_stereo_test, label_stereo_train, label_stereo_test = cross_validation.train_test_split(data_stereo.drop(['label'], axis=1),\
                                                                                                               data_stereo[['label']],\
                                                                                                               test_size = 0.3)

In [27]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clf_stereo = RandomForestClassifier(n_estimators=1000, 
                               max_features='auto', 
                               max_depth=None, 
                               bootstrap=False, 
                               class_weight='balanced',
                               n_jobs = 3,
                               criterion = 'gini',
                               min_samples_leaf = 1)
#clf_y = GradientBoostingClassifier(n_estimators=1000, 
#                                   learning_rate=0.01, 
#                                   max_depth=6, 
#                                   max_features='auto', 
#                                   subsample=0.5,
#                                   min_samples_split=10)

clf_stereo.fit(data_stereo_train.values, label_stereo_train.values)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=3,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
clf_stereo.score(data_stereo_test.values, label_stereo_test.values)

0.98352941176470587

In [29]:
clf_stereo.feature_importances_

array([ 0.07173046,  0.34248329,  0.25555817,  0.2264949 ,  0.06305341,
        0.03850642,  0.00217335])

In [40]:
import cPickle as pickle

clf_stereo_pickle = open('clf_stereo.pkl', 'wb')
pickle.dump(clf_stereo, clf_stereo_pickle)
clf_stereo_pickle.close()

In [41]:
import cPickle as pickle

clf_stereo_pickle2 = open('clf_stereo.pkl', 'rb')
clf_stereo = pickle.load(clf_stereo_pickle2)
clf_stereo_pickle2.close()

In [45]:
data_stereo.head()

,min_dist,max_dist,score_std,score,slope,intercept,label,len
0,0.007302,0.334408,0.125418,0.196261,-0.015163,63.108224,1,10
1,1.259619,9.913027,2.782271,4.891600,0.627255,-1582.378602,0,10
2,0.017426,0.269795,0.104010,0.158463,0.023567,84.801992,1,9
3,1.058307,43.673853,14.277642,18.732651,-0.493514,1406.961035,0,9
4,0.031446,0.460475,0.194752,0.275425,-0.006853,-93.787694,1,9


In [48]:
clf_stereo.predict_proba(data_stereo.drop(['label'], axis=1).iloc[0])

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


1.0

In [46]:
data_stereo.iloc[0]

min_dist      0.007302
max_dist      0.334408
score_std     0.125418
score         0.196261
slope        -0.015163
intercept    63.108224
label         1.000000
len          10.000000
Name: 0, dtype: float64

In [49]:
import cPickle as pickle

file_pickle = open('./Data/reco_tracks_for_pattern_recognition.pkl', 'rb')
reconstructible_events = pickle.load(file_pickle)
file_pickle.close()

In [52]:
reconstructible_events.keys()

[513,
 2,
 515,
 516,
 1541,
 518,
 7,
 10,
 2059,
 1036,
 614,
 1550,
 15,
 856,
 1042,
 1557,
 23,
 885,
 27,
 2077,
 768,
 1057,
 549,
 1439,
 39,
 1066,
 174,
 1458,
 558,
 2095,
 48,
 189,
 50,
 2099,
 53,
 692,
 1546,
 62,
 1600,
 2113,
 1090,
 694,
 1606,
 1607,
 1548,
 74,
 781,
 184,
 257,
 84,
 85,
 86,
 1111,
 1693,
 2138,
 1528,
 1116,
 1629,
 606,
 1381,
 608,
 97,
 612,
 1637,
 102,
 1129,
 106,
 1383,
 2157,
 622,
 2493,
 1136,
 625,
 1138,
 2164,
 1142,
 1655,
 1848,
 122,
 126,
 129,
 642,
 131,
 1668,
 1157,
 134,
 135,
 648,
 876,
 651,
 1677,
 654,
 143,
 294,
 2204,
 1683,
 793,
 1691,
 1180,
 157,
 670,
 159,
 2208,
 161,
 1222,
 1190,
 1704,
 686,
 2224,
 712,
 449,
 179,
 1204,
 1717,
 1206,
 381,
 2232,
 1722,
 1724,
 1213,
 2239,
 707,
 2244,
 2246,
 893,
 1224,
 1200,
 842,
 719,
 1232,
 211,
 2260,
 1238,
 215,
 2264,
 1242,
 1243,
 1756,
 221,
 1758,
 2271,
 736,
 1761,
 1762,
 1251,
 2277,
 1934,
 1256,
 1769,
 2282,
 747,
 978,
 1262,
 1775,
 1265,
 2290,